<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-1:-Preprocessing" data-toc-modified-id="Part-1:-Preprocessing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Part 1: Preprocessing</a></span></li><li><span><a href="#Part-2:-Make-the-ANN" data-toc-modified-id="Part-2:-Make-the-ANN-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part 2: Make the ANN</a></span></li><li><span><a href="#Part-3:-Prediction" data-toc-modified-id="Part-3:-Prediction-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Part 3: Prediction</a></span></li><li><span><a href="#Part-4:-Evaluating,-Improving-and-Tuning-the-ANN" data-toc-modified-id="Part-4:-Evaluating,-Improving-and-Tuning-the-ANN-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Part 4: Evaluating, Improving and Tuning the ANN</a></span><ul class="toc-item"><li><span><a href="#Dropout" data-toc-modified-id="Dropout-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Dropout</a></span></li><li><span><a href="#Parameter-tuning-using-Grid-Search" data-toc-modified-id="Parameter-tuning-using-Grid-Search-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Parameter tuning using Grid Search</a></span></li></ul></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image, display

df = pd.read_csv('../data/csv/Churn_Modelling.csv')
X = df.iloc[:,3:-1].values
y = df.iloc[:, -1].values

print(df.shape)
print(X[0])
print(y[0])
df.head()

(10000, 14)
[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
1


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# Part 1: Preprocessing

In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# encode geography country names
le_geo = LabelEncoder()
X[:, 1] = le_geo.fit_transform(X[:, 1])

# encode gender male female
le_gen = LabelEncoder()
X[:, 2] = le_gen.fit_transform(X[:, 2])

ohe = OneHotEncoder(categorical_features=[1])
X = ohe.fit_transform(X).toarray()
X = X[:, 1:] # do not take first dummy variable

print(X.shape)
print(X[0])
df2 = pd.DataFrame(X)

display(df.iloc[:,3:-1].head(20), df2.head(20))
# we can see france is 0,0  and female is 0 categorical features.

(10000, 11)
[0.0000000e+00 0.0000000e+00 6.1900000e+02 0.0000000e+00 4.2000000e+01
 2.0000000e+00 0.0000000e+00 1.0000000e+00 1.0000000e+00 1.0000000e+00
 1.0134888e+05]


/Users/poudel/miniconda3/envs/tf/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/poudel/miniconda3/envs/tf/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
5,645,Spain,Male,44,8,113755.78,2,1,0,149756.71
6,822,France,Male,50,7,0.00,2,1,1,10062.80
7,376,Germany,Female,29,4,115046.74,4,1,0,119346.88
8,501,France,Male,44,4,142051.07,2,0,1,74940.50
9,684,France,Male,27,2,134603.88,1,1,1,71725.73


,0,1,2,3,4,5,6,7,8,9,10
0,0.0,0.0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0,101348.88
1,0.0,1.0,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0,112542.58
2,0.0,0.0,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0,113931.57
3,0.0,0.0,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0,93826.63
4,0.0,1.0,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0,79084.10
5,0.0,1.0,645.0,1.0,44.0,8.0,113755.78,2.0,1.0,0.0,149756.71
6,0.0,0.0,822.0,1.0,50.0,7.0,0.00,2.0,1.0,1.0,10062.80
7,1.0,0.0,376.0,0.0,29.0,4.0,115046.74,4.0,1.0,0.0,119346.88
8,0.0,0.0,501.0,1.0,44.0,4.0,142051.07,2.0,0.0,1.0,74940.50
9,0.0,0.0,684.0,1.0,27.0,2.0,134603.88,1.0,1.0,1.0,71725.73


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape

(8000, 11)

In [4]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2: Make the ANN

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix

# Build classifier
clf = Sequential()
clf.add(Dense(units=6, kernel_initializer='uniform', activation='relu',input_dim=11))
clf.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
clf.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
clf.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

# fitting
clf.fit(X_train,y_train, batch_size=10, epochs=100)

# prediction
y_pred = clf.predict(X_test)
y_pred = (y_pred > 0.5)

# confusion matrix and accuracy
cm = confusion_matrix(y_test, y_pred)
acc = (cm[0,0] + cm[1,1]) / np.sum(cm.flatten())
print(acc)

Using TensorFlow backend.


Epoch 1/100
8000/8000 [==============================] - 1s 127us/step - loss: 0.4815 - acc: 0.7957
Epoch 2/100
8000/8000 [==============================] - 1s 97us/step - loss: 0.4256 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 92us/step - loss: 0.4202 - acc: 0.8062: 0s - loss: 0.4184 - acc: 0.80
Epoch 4/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.4174 - acc: 0.8257: 0s - loss: 0.4198 - acc: 
Epoch 5/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4155 - acc: 0.8290
Epoch 6/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4136 - acc: 0.8312
Epoch 7/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4121 - acc: 0.8302
Epoch 8/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4107 - acc: 0.8329
Epoch 9/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4103 - acc: 0.8329
Epoch 10/100
8000/8000 [==========================

8000/8000 [==============================] - 1s 92us/step - loss: 0.4004 - acc: 0.8355: 0s - loss: 0.4021 - acc: 0.833
Epoch 79/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.3998 - acc: 0.8345
Epoch 80/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4002 - acc: 0.8339
Epoch 81/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4004 - acc: 0.8352
Epoch 82/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4001 - acc: 0.8362
Epoch 83/100
8000/8000 [==============================] - 1s 91us/step - loss: 0.4004 - acc: 0.8347
Epoch 84/100
8000/8000 [==============================] - 1s 92us/step - loss: 0.4003 - acc: 0.8355: 0s - loss: 0.4050 - a
Epoch 85/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4002 - acc: 0.8350
Epoch 86/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.4003 - acc: 0.8346
Epoch 87/100
8000/8000 [==============================] - 

# Part 3: Prediction

In [6]:
y_pred = clf.predict(X_test)
y_pred = (y_pred > 0.5)

In [7]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1543,   52],
       [ 264,  141]])

In [8]:
acc = (cm[0,0] + cm[1,1]) / np.sum(cm.flatten())
acc

0.842

# Part 4: Evaluating, Improving and Tuning the ANN

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

def build_classifier():
    clf = Sequential()
    clf.add(Dense(units=6, kernel_initializer='uniform', activation='relu',input_dim=11))
    clf.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    clf.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    clf.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return clf

clf = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)
acc = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=10, n_jobs=-1)

mean = acc.mean()
std= acc.std()

print(mean, std)

0.8441249937564134 0.01204224272181121


##  Dropout
dropout regularization to reduce overfitting if needed

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from keras.layers import Dropout

# Build classifier
clf = Sequential()

clf.add(Dense(units=6, kernel_initializer='uniform', activation='relu',input_dim=11))
clf.add(Dropout(0.1))

clf.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
clf.add(Dropout(0.1))

clf.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
clf.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

# fitting
clf.fit(X_train,y_train, batch_size=10, epochs=100)

# prediction
y_pred = clf.predict(X_test)
y_pred = (y_pred > 0.5)

# confusion matrix and accuracy
cm = confusion_matrix(y_test, y_pred)
acc = (cm[0,0] + cm[1,1]) / np.sum(cm.flatten())
print(acc)

Epoch 1/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.4992 - acc: 0.7955 1s - loss: 0.5912 - a
Epoch 2/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4382 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4355 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4326 - acc: 0.8110
Epoch 5/100
8000/8000 [==============================] - 1s 102us/step - loss: 0.4281 - acc: 0.8229 0s - loss: 0.4275 - acc: 0.823
Epoch 6/100
8000/8000 [==============================] - 1s 101us/step - loss: 0.4300 - acc: 0.8212
Epoch 7/100
8000/8000 [==============================] - 1s 100us/step - loss: 0.4252 - acc: 0.8275
Epoch 8/100
8000/8000 [==============================] - 1s 102us/step - loss: 0.4232 - acc: 0.8284 0s - loss: 0.4225 - acc: 0.
Epoch 9/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4210 - acc: 0.8247
Epoch 10/100
8000/8

8000/8000 [==============================] - 1s 100us/step - loss: 0.4089 - acc: 0.8322
Epoch 77/100
8000/8000 [==============================] - 1s 100us/step - loss: 0.4103 - acc: 0.8361
Epoch 78/100
8000/8000 [==============================] - 1s 107us/step - loss: 0.4078 - acc: 0.8372
Epoch 79/100
8000/8000 [==============================] - 1s 102us/step - loss: 0.4081 - acc: 0.8355
Epoch 80/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4133 - acc: 0.8324
Epoch 81/100
8000/8000 [==============================] - 1s 99us/step - loss: 0.4093 - acc: 0.8366: 0s - loss: 0.4105 - acc: 
Epoch 82/100
8000/8000 [==============================] - 1s 100us/step - loss: 0.4088 - acc: 0.8362
Epoch 83/100
8000/8000 [==============================] - 1s 125us/step - loss: 0.4113 - acc: 0.8352
Epoch 84/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4083 - acc: 0.8352
Epoch 85/100
8000/8000 [==============================] - 1s 100us/step - loss

## Parameter tuning using Grid Search

In [11]:
%%writefile ann_parameter_tuning.py

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    clf = Sequential()
    clf.add(Dense(units=6, kernel_initializer='uniform', activation='relu',input_dim=11))
    clf.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    clf.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    clf.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])
    return clf

clf = KerasClassifier(build_fn=build_classifier)

parameters = {'batch_size': [25, 32],
             'epochs': [100,500],
             'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator=clf,
                          param_grid = parameters,
                          scoring='accuracy',
                          cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_acc = grid_search.best_score_

print(best_parameters, best_acc)

Overwriting ann_parameter_tuning.py
